In [ ]:
using ITensors, ITensorMPS, PythonPlot

# Global variables to store results
global C1zz = nothing
global L = 20

# Main computation
let
    # Create L Spin-1/2 Indices
    sites = siteinds("S=1/2", L)

    # Create the Hamiltonian of the Ising chain at its self-dual critical point from the O'Brien and Fendley paper.
    osI = OpSum()
    for j in 1:(L - 1)
        osI -= "X", j 
        osI -= "Z", j, "Z", j + 1
    end
    osI -= "X", L
    osI -= "Z", L, "Z", 1
    HI = MPO(osI, sites)

    # Create the Hamiltonian of the three-spin interaction from the O'Brien and Fendley paper.
    os3 = OpSum()
    for j in 1:(L - 2)
        os3 += "X", j, "Z", j + 1, "Z", j + 2
        os3 += "Z", j, "Z", j + 1, "X", j + 2
    end
    os3 += "X", L - 1, "Z", L, "Z", 1
    os3 += "X", L, "Z", 1, "Z", 2
    os3 += "Z", L - 1, "Z", L, "X", 1
    os3 += "Z", L, "Z", 1, "X", 2
    H3 = MPO(os3, sites)

    # Coupling coefficients for Exact G.S.
        lambdaI = 1
        lambda3 = 1
    
    # Coupling coefficients for TCI such that lambda3 / lambdaI = 0.856
        # lambdaI = 0.759685
        # lambda3 = 0.650291
    
    # Energy offset
    EO = L * (lambdaI^2 + lambda3^2) / lambda3

    # Adding the energy offset to just one site
    osEO = OpSum()
        osEO += EO, "Id", 1 # "Id" is the identity operator
    E0 = MPO(osEO, sites)

    # Full Hamiltonian from paper
    H = 2 * lambdaI * HI + lambda3 * H3 + E0

    # Create an initial random matrix product state
    psi0 = random_mps(sites)
    nsweeps = 20

    maxdim = [10, 33, 66, 133, 200]
    mindim = maxdim
    cutoff = 1.0e-10

    # Run the DMRG algorithm, returning energy
    # (dominant eigenvalue) and optimized MPS
    println("Maxdim per sweep: ", maxdim)
    
    energy1, psi1 = dmrg(H, psi0; nsweeps, mindim, maxdim, cutoff)
    #energy2, psi2 = dmrg(H, [psi1], psi0; nsweeps, mindim, maxdim, cutoff, weight = 100)
    #energy3, psi3 = dmrg(H, [psi1, psi2], psi0; nsweeps, mindim, maxdim, cutoff, weight = 100)
    #energy4, psi4 = dmrg(H, [psi1, psi2, psi3], psi0; nsweeps, mindim, maxdim, cutoff, weight = 100)
    
    println("Final Energy 1 = $energy1")
    #println("Final Energy 2 = $energy2")
    #println("Final Energy 3 = $energy3")
    #println("Final Energy 4 = $energy4")
    
    println("Overlap 1,1 = $(inner(psi1, psi1))")
    #println("Overlap 1,2 = $(inner(psi1, psi2))")
    #println("Overlap 1,3 = $(inner(psi1, psi3))")
    #println("Overlap 1,4 = $(inner(psi1, psi4))")
    #println("Overlap 2,3 = $(inner(psi2, psi3))")
    #println("Overlap 2,4 = $(inner(psi2, psi4))")
    #println("Overlap 3,4 = $(inner(psi3, psi4))")

    # Compute Z-Z correlation matrix for the first G.S. and store in global
    global C1zz = correlation_matrix(psi1, "Z", "Z")

    nothing
end

Maxdim per sweep: [10, 33, 66, 133, 200]
After sweep 1 energy=1.8913049829167612  maxlinkdim=4 maxerr=4.42E-16 time=0.019
After sweep 2 energy=0.0006050918659898445  maxlinkdim=16 maxerr=4.70E-16 time=0.047
After sweep 2 energy=0.0006050918659898445  maxlinkdim=16 maxerr=4.70E-16 time=0.047
After sweep 3 energy=7.985735741827351e-7  maxlinkdim=64 maxerr=6.06E-16 time=0.213
After sweep 3 energy=7.985735741827351e-7  maxlinkdim=64 maxerr=6.06E-16 time=0.213
After sweep 4 energy=1.4891538714681968e-9  maxlinkdim=133 maxerr=5.63E-16 time=1.898
After sweep 4 energy=1.4891538714681968e-9  maxlinkdim=133 maxerr=5.63E-16 time=1.898
After sweep 5 energy=2.4141304703801448e-12  maxlinkdim=200 maxerr=1.23E-15 time=2.868
After sweep 5 energy=2.4141304703801448e-12  maxlinkdim=200 maxerr=1.23E-15 time=2.868
After sweep 6 energy=-4.121957629714231e-13  maxlinkdim=200 maxerr=1.59E-16 time=3.211
After sweep 6 energy=-4.121957629714231e-13  maxlinkdim=200 maxerr=1.59E-16 time=3.211
After sweep 7 energy

In [ ]:
using PythonPlot

# Get the absolute values of correlations
abs_C1zz = abs.(C1zz[1, :])
    
x = collect(1:length(abs_C1zz))
plot(x, abs_C1zz, marker="o", linestyle="-", color="tab:blue", linewidth=1.5)

xlabel("Site L")
title("Absolute Value of Z-Z Correlation Between Site 1 and Site L")
grid(true, which="major", alpha=0.5)
grid(true, which="minor", alpha=0.2)
    
# Adjusting graph to show the data better
ymax = maximum(abs_C1zz)
ymin = minimum(abs_C1zz)
ylim(ymin - 0.1, ymax + 0.1)
minorticks_on()
    
# Print max and min values for reference
println("Maximum correlation: ", maximum(abs_C1zz))
println("Minimum correlation: ", minimum(abs_C1zz))


LoadError: UndefVarError: `C1zz` not defined in `Main`
Suggestion: check for spelling errors or missing imports.